In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from cloth_tools.drake.building import add_dual_ur5e_and_table_to_builder, add_meshcat_to_builder, finish_build
from cloth_tools.ompl.dual_arm_planner import DualArmOmplPlanner
from cloth_tools.drake.visualization import publish_dual_arm_joint_path
from cloth_tools.urdf.primitives import cylinder_urdf
from pydrake.math import RigidTransform, RollPitchYaw

In [ ]:
start_joints_left = np.deg2rad([0, -90, -90, -90, 90, 0])
start_joints_right = np.deg2rad([-136, -116, -110, -133, 40, 0])

home_joints_left = np.deg2rad([180, -135, 95, -50, -90, -90])
home_joints_right = np.deg2rad([-180, -45, -95, -130, 90, 90])

lowest_joints_left = np.deg2rad([180, -90, 30, -120, -90, -90])
lowest_goal_joints_right = np.deg2rad([-135, -110, -80, -175, 45, 0])

In [ ]:
# Creating the default scene
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_left_index, arm_right_index = add_dual_ur5e_and_table_to_builder(robot_diagram_builder)
diagram, context = finish_build(robot_diagram_builder, meshcat)

plant = diagram.plant()
gripper_left_index = plant.GetModelInstanceByName("gripper_left")
gripper_right_index = plant.GetModelInstanceByName("gripper_right")

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_left_index, arm_right_index, gripper_left_index, gripper_right_index],
    edge_step_size=0.125, # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding = 0.005,
    self_collision_padding = 0.005,
)

is_state_valid_default_fn = collision_checker.CheckConfigCollisionFree

In [ ]:
collision_checker.CheckConfigCollisionFree(np.concatenate([home_joints_left, home_joints_right]))

In [ ]:
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

plant.SetPositions(plant_context, arm_left_index, start_joints_left)
plant.SetPositions(plant_context, arm_right_index, start_joints_right)
diagram.ForcedPublish(context)

In [ ]:
# Creating the scene with a cylinder obstacles
robot_diagram_builder = RobotDiagramBuilder()
meshcat = add_meshcat_to_builder(robot_diagram_builder)
arm_left_index, arm_right_index = add_dual_ur5e_and_table_to_builder(robot_diagram_builder)

cylinder_urdf_str = cylinder_urdf(0.55, 0.1)

plant = robot_diagram_builder.plant()
parser = robot_diagram_builder.parser()
cylinder_index = parser.AddModelsFromString(cylinder_urdf_str, "urdf")[0]
cylinder_frame = plant.GetFrameByName("base_link", cylinder_index)
cylinder_transform = RigidTransform(p=[0.45, 0.1, 0.65])
plant.WeldFrames(plant.world_frame(), cylinder_frame, cylinder_transform)

diagram, context = finish_build(robot_diagram_builder, meshcat)

gripper_left_index = plant.GetModelInstanceByName("gripper_left")
gripper_right_index = plant.GetModelInstanceByName("gripper_right")

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_left_index, arm_right_index, gripper_left_index, gripper_right_index],
    edge_step_size=0.125, # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding = 0.005,
    self_collision_padding = 0.005,
)

is_state_valid_cylinder_fn = collision_checker.CheckConfigCollisionFree

In [ ]:
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

plant.SetPositions(plant_context, arm_left_index, lowest_joints_left)
plant.SetPositions(plant_context, arm_right_index, lowest_goal_joints_right)
diagram.ForcedPublish(context)

In [ ]:
is_state_valid_cylinder_fn(np.concatenate([lowest_joints_left, lowest_goal_joints_right]))

In [ ]:
is_state_valid_cylinder_fn(np.concatenate([lowest_joints_left, home_joints_right]))

In [ ]:
planner = DualArmOmplPlanner(is_state_valid_cylinder_fn, max_planning_time=10.0)

In [ ]:
path = planner.plan_to_joint_configuration(lowest_joints_left, home_joints_right, None, lowest_goal_joints_right)

In [ ]:
if path is None:
    print("No path found")
else:
    publish_dual_arm_joint_path(path, 5.0, meshcat, diagram, context, arm_left_index, arm_right_index)